<a href="https://colab.research.google.com/github/Ahmedfull02/MICROSOFT-SECURITY-INCIDENT-PREDICTION-USING-MULTI-AGENT-SYSTEM/blob/master/Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
microsoft_microsoft_security_incident_prediction_path = kagglehub.dataset_download('Microsoft/microsoft-security-incident-prediction')

print('Data source import complete.')


# MICROSOFT SECURITY INCIDENT PREDICTION

BY: MARINA J. CARRANZA SÁNCHEZ & CARLOTA DE LA VEGA SORIANO

# 1. Preparación del entorno

## 1.1. Importaciones

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

## 1.2. Estilo de las gráficas

In [ ]:
def style_and_save(
    ax,
    title='',
    xlabel='',
    ylabel='',
    filename='plot.png',
    title_pad=20,
    xlabel_pad=15,
    ylabel_pad=15,
    xtick_rotation=0,
    ytick_rotation=0,
    grid_axis='y',
    fontsize=11,
    show_legend=True
):
    ax.set_title(title, fontsize=12, fontweight='bold', pad=title_pad)
    ax.set_xlabel(xlabel, fontsize=10, fontweight='semibold', labelpad=xlabel_pad)
    ax.set_ylabel(ylabel, fontsize=10, fontweight='semibold', labelpad=ylabel_pad)
    ax.tick_params(axis='x', labelrotation=xtick_rotation, labelsize=fontsize)
    ax.tick_params(axis='y', labelrotation=ytick_rotation, labelsize=fontsize)
    ax.grid(axis=grid_axis, linestyle='--', alpha=0.5)
    if show_legend:
        ax.legend(loc='lower right', fontsize=10)
    plt.tight_layout()
    plt.savefig(filename, dpi=300)
    plt.show()

In [ ]:
def create_count_plot(
    df,
    column,
    title='',
    xlabel='',
    ylabel='',
    filename='count_plot.png',
    order=None,
    xtick_rotation=0,
    annotate_values=False,
    figsize=(8, 5)
):
    plt.figure(figsize=figsize)
    ax = sns.countplot(
        data=df[df[column].notnull()],
        x=column,
        order=order,
        edgecolor='gray',
        palette='pastel'
    )

    if annotate_values:
        for bar in ax.patches:
            height = bar.get_height()
            ax.annotate(f'{height:,}',
                        (bar.get_x() + bar.get_width() / 2., height),
                        ha='center', va='bottom', fontsize=10, fontweight='semibold')

    style_and_save(
        ax,
        title=title,
        xlabel=xlabel,
        ylabel=ylabel,
        filename=filename,
        xtick_rotation=xtick_rotation
    )


In [ ]:
def create_stacked_barh(
    df,
    title='',
    xlabel='',
    ylabel='',
    filename='stacked_barh.png',
    colors=None,
    figsize=(10, 6),
    annotate_values=False
):
    if colors is None:
        colors = sns.color_palette('pastel')[:df.shape[1]]

    ax = df.plot(
        kind='barh',
        stacked=True,
        figsize=figsize,
        color=colors,
        edgecolor='gray'
    )

    if annotate_values:
        for container in ax.containers:
            for bar in container:
                width = bar.get_width()
                if width > 0:
                    ax.text(
                        bar.get_x() + width / 2,
                        bar.get_y() + bar.get_height() / 2,
                        f'{int(width):,}',
                        ha='center', va='center',
                        fontsize=9
                    )

    style_and_save(
        ax,
        title=title,
        xlabel=xlabel,
        ylabel=ylabel,
        filename=filename,
        grid_axis='x'
    )

## 1.3. Carga del dataset

In [ ]:
df = pd.read_csv('/kaggle/input/microsoft-security-incident-prediction/GUIDE_Train.csv')

# 2. Analisis exploratorio

## 2.1. Definición del dataset

### 2.1.1. Variables (columnas)

In [ ]:
df.info()

Here is the description for the features, as mentioned in the source paper titled [**AI-Driven Guided Response for Security Operation Centers with Microsoft Copilot for Security**](https://arxiv.org/pdf/2407.09017).
| **Feature**            | **Description**                                                                                               |
| ---------------------- | ------------------------------------------------------------------------------------------------------------- |
| **Id**                 | Unique identifier for each OrgId-IncidentId combination, ensuring global uniqueness of each record.           |
| **OrgId**              | Identifier for the organization where the incident or alert originated, used to separate data across tenants. |
| **IncidentId**         | Unique identifier assigned by the organization to each tracked or investigated security incident.             |
| **AlertId**            | Unique identifier for an individual alert generated by a detection system or security tool.                   |
| **Timestamp**          | Date and time when the alert was created or logged in the system, in UTC format.                              |
| **DetectorId**         | Identifier of the detection engine or system that generated the alert.                                        |
| **AlertTitle**         | Short descriptive title summarizing the nature of the alert.                                                  |
| **Category**           | Broad classification or category of the alert, indicating the type of detected threat or issue.               |
| **MitreTechniques**    | List of MITRE ATT\&CK techniques associated with the alert, describing the tactics involved.                  |
| **IncidentGrade**      | Severity level assigned to the incident by the Security Operations Center after review.                       |
| **ActionGrouped**      | High-level description of the remediation or response action taken.                                           |
| **ActionGranular**     | Detailed description of the remediation action taken, specifying exact measures.                              |
| **EntityType**         | Type of entity involved in the alert.                                                                         |
| **EvidenceRole**       | Role of the evidence within the investigation.                                                                |
| **Roles**              | Additional metadata or labels indicating other roles played by the entity in the alert or investigation.      |
| **DeviceId**           | Unique identifier assigned to the device involved in the alert.                                               |
| **DeviceName**         | Human-readable name of the device involved in the alert.                                                      |
| **Sha256**             | SHA-256 hash of the file involved in the alert.                                                               |
| **IpAddress**          | IP address associated with the alert.                                                                         |
| **Url**                | URL involved in the alert.                                                                                    |
| **AccountSid**         | Identifier for an on-premises account linked to the alert or incident.                                        |
| **AccountUpn**         | User Principal Name of the account involved in the alert.                                                     |
| **AccountObjectId**    | Entra ID object identifier for the user or account involved.                                                  |
| **AccountName**        | Username of the on-premises account involved in the alert.                                                    |
| **NetworkMessageId**   | Organization-level unique identifier for an email message involved in the alert.                              |
| **EmailClusterId**     | Identifier for a cluster or group of related email messages involved in the incident.                         |
| **RegistryKey**        | Windows registry key involved in the alert.                                                                   |
| **RegistryValueName**  | Name of the registry value that was modified, created, or accessed.                                           |
| **RegistryValueData**  | Data stored in the registry value at the time of the alert.                                                   |
| **ApplicationId**      | Unique identifier for an application involved in the alert.                                                   |
| **ApplicationName**    | Name of the application involved in the alert or incident.                                                    |
| **OAuthApplicationId** | Identifier for the OAuth application involved.                                                                |
| **ThreatFamily**       | Name of the malware family detected as part of the alert.                                                     |
| **FileName**           | Name of the file involved in the alert.                                                                       |
| **FolderPath**         | Path where the file was located at the time of the alert.                                                     |
| **ResourceIdName**     | Name of the Azure resource involved in the alert or investigation.                                            |
| **ResourceType**       | Type of Azure resource involved.                                                                              |
| **OSFamily**           | Family of the operating system running on the device involved.                                                |
| **OSVersion**          | Version of the operating system running on the device involved.                                               |
| **AntispamDirection**  | Direction of the email traffic as processed by the antispam system.                                           |
| **SuspicionLevel**     | Assigned level of suspicion for the alert or entity.                                                          |
| **LastVerdict**        | Final determination or classification for the alert after review.                                             |
| **CountryCode**        | ISO country code indicating the geographic location associated with the alert.                                |
| **State**              | State or province name associated with the geographic location of the alert evidence.                         |
| **City**               | City name associated with the geographic location of the alert evidence.                                      |


### 2.1.2. Filas

In [ ]:
total_rows = len(df)
print(total_rows)

## 2.2. Elección de la variable objetivo

In [ ]:
variables_candidatas = ['IncidentGrade', 'SuspicionLevel', 'LastVerdict', 'Category']

for col in variables_candidatas:
    print(f"\n📌 Variable: {col}")
    print("-" * 40)
    print(f"Valores únicos:\n{df[col].unique()}")
    print(f"\nNúmero de valores nulos: {df[col].isnull().sum()} de {len(df)} registros totales")
    print("=" * 60)

In [ ]:
variables = ['IncidentGrade', 'SuspicionLevel', 'LastVerdict', 'Category']

# Cálculo de nulos y no nulos

data = {
    'Not Null': [df[var].notnull().sum() for var in variables],
    'Null': [df[var].isnull().sum() for var in variables]
}

# Creación del dataframe

df_nulls = pd.DataFrame(data, index=variables)

# Gráfico

create_stacked_barh(
    df=df_nulls,
    title='Valores Nulos vs No Nulos por variable candidata',
    xlabel='Número de registros',
    ylabel='Variable',
    filename='valores_nulos_por_variable.png',
    annotate_values=False
)


## 2.3. Distribución de la variable objetivo

In [ ]:
create_count_plot(
    df=df,
    column='IncidentGrade',
    title='Distribución de la variable objetivo: IncidentGrade',
    xlabel='Clase',
    ylabel='Cantidad de registros',
    filename='distribucion_variable_objetivo.png',
    order=df['IncidentGrade'].value_counts().index
)

## 2.4. Visualización de valores nulos

In [ ]:
# Contar los valores nulos por columna

missing_values = df.isnull().sum()
missing_values = missing_values[missing_values > 0]
missing_values = missing_values.sort_values()

# Visualizar

plt.figure(figsize=(12, 6))
missing_values.plot(kind='barh', color='skyblue')
plt.title('Número de valores faltantes por columna')
plt.xlabel('Cantidad de nulos')
plt.ylabel('Columnas')
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.show()

# 3. Preprocesamiento

## 3.1. Limpieza de datos

### 3.1.1. Eliminar filas/columnas por valores nulos

In [ ]:
def viewRemainingColumns():
    numeric_columns = df.select_dtypes(include=['float64', 'int64', 'int32']).columns
    string_columns = df.select_dtypes(include=['object']).columns

    # Imprimir los nombres de las columnas
    print("Columnas numéricas:")
    print(numeric_columns)
    print(f"Total de columnas numéricas: {len(numeric_columns)}\n")

    print("Columnas de tipo string:")
    print(string_columns)
    print(f"Total de columnas de tipo string: {len(string_columns)}")

In [ ]:
# Eliminar columnas con más del 70% de valores nulos

columns_with_high_null = []

for column in df.columns:
    if df[column].isnull().sum() / total_rows > 0.7:
        columns_with_high_null.append(column)

print("Columnas con más del 70% de valores nulos:", columns_with_high_null)
print(f"Total de columnas a eliminar: {len(columns_with_high_null)}\n")

df = df.drop(columns=columns_with_high_null)

In [ ]:
viewRemainingColumns()

In [ ]:
# Borrar filas cuyo Incident Grade sea nulo para no inventarnos la clase objetivo

df = df.dropna(subset=['IncidentGrade'])

total_rows_after_delete = len(df)
print(f"{round((total_rows_after_delete / total_rows * 100), 2)}%")

In [ ]:
# Actualizar número total de filas

total_rows = total_rows_after_delete

### 3.1.2. Imputación de valores nulos


In [ ]:
# Imputar los valores (con la moda) de MitreTechniques

print(df['MitreTechniques'].isnull().sum())

mode = df['MitreTechniques'].mode()[0]

df.fillna({'MitreTechniques': mode}, inplace=True)

print(df['MitreTechniques'].isnull().sum())

### 3.1.3. Eliminar columnas irrelevantes

In [ ]:
def format_percentage(num_uniques, total_rows):
    percentage = (num_uniques / total_rows) * 100
    rounded = round(percentage, 2)
    return "<0.01%" if rounded == 0.0 and num_uniques > 0 else f"{rounded:.2f}%"

In [ ]:
def get_uniques_list(cols):
    return sorted(
        [(col, df[col].nunique()) for col in cols],
        key=lambda x: x[1],
        reverse=True)


In [ ]:
# Comprobar cuántos valores son únicos en los registros (ID)

numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
string_cols = df.select_dtypes(include=['object']).columns

numeric_uniques = get_uniques_list(numeric_cols)
string_uniques = get_uniques_list(string_cols)

print("\nValores únicos para cada columna numérica:\n")
print(f"{'Columna':<30} {'Únicos':<10} {'% sobre total'}")
print("-" * 55)
for col, num_uniques in numeric_uniques:
    percentage_str = format_percentage(num_uniques, total_rows)
    print(f"{col:<30} {num_uniques:<10} {percentage_str}")

print("\n\nValores únicos para cada columna categórica:\n")
print(f"{'Columna':<30} {'Únicos':<10} {'% sobre total'}")
print("-" * 55)
for col, num_uniques in string_uniques:
    percentage_str = format_percentage(num_uniques, total_rows)
    print(f"{col:<30} {num_uniques:<10} {percentage_str}")


In [ ]:
# Listado de columnas irrelevantes para la matriz de correlación
# Selección en función de output anterior (>3%)
irrelevant_columns = ['AlertId', 'Id', 'AccountUpn', 'IncidentId', 'NetworkMessageId',
                     'AccountName', 'AccountSid', 'AccountObjectId', 'IpAddress']

df = df.drop(columns=irrelevant_columns)

viewRemainingColumns()

## 3.2. Codificación de variables categóricas

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Seleccionar las columnas de tipo 'object', excluyendo 'Timestamp'
encode_label = df.select_dtypes([object]).columns
encode_label = [col for col in encode_label if col != 'Timestamp']

print(encode_label)

# Diccionario para almacenar los codificadores
label_encoders = {}

# Aplicar LabelEncoder a cada columna categórica, excepto 'Timestamp'
for col in encode_label:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le


## 3.3. Tratamiento de fechas

### 3.3.1. Conversión de fechas


In [ ]:
# Procesar DateTime

unique_lengths = set(df['Timestamp'].map(len))
print(unique_lengths)

df['Timestamp'] = pd.to_datetime(df['Timestamp'])

df['Year'] = df['Timestamp'].dt.year
df['Month'] = df['Timestamp'].dt.month
df['Day'] = df['Timestamp'].dt.day
df['Hour'] = df['Timestamp'].dt.hour
df['Minute'] = df['Timestamp'].dt.minute
df['Second'] = df['Timestamp'].dt.second
df['Weekday'] = df['Timestamp'].dt.weekday  # 0 = Monday, 6 = Sunday

### 3.3.2. Agrupamiento de fechas

In [ ]:
## Día: 0 | Noche: 1

def asign_period(hour):
    if 7 <= hour < 19:
        return 0
    else:
        return 1


## Primavera: 0 | Verano: 1 | Otoño: 2 | Invierno: 3

def asign_season(month):
    if 3 <= month <= 5:
        return 0
    elif 6 <= month <= 8:
        return 1
    elif 9 <= month <= 11:
        return 2
    else:
        return 3


## Lectivo: 0 | No lectivo: 1

def asign_holidays(weekday):
    if 0 <= weekday < 5:
        return 0
    else:
        return 1

In [ ]:
# Por franja horaria
df['Period'] = df['Hour'].apply(asign_period)

# Por estaciones
df['Season'] = df['Month'].apply(asign_season)

# Por lectividad
df['Holiday'] = df['Weekday'].apply(asign_holidays)

In [ ]:
viewRemainingColumns()

### 3.3.3. Eliminación de atributos de fechas

In [ ]:
print("Valores únicos para el año:", len(df['Year'].unique())) # Se puede eliminar esta variable

In [ ]:
df = df.drop(columns=['Year', 'Month', 'Weekday', 'Day', 'Hour', 'Minute', 'Second'])
df = df.drop(columns=['Timestamp'])

In [ ]:
viewRemainingColumns()

## 3.4. Análisis de correlación

### 3.4.1. Matriz de correlación

In [ ]:
# Calcular la matriz de correlación
correlation_matrix = df.corr()

# Visualización de la matriz de correlación
plt.figure(figsize=(18, 16))
ax = sns.heatmap(correlation_matrix, annot=False, cmap="coolwarm", linewidths=0.5)
plt.title("Matriz de correlación")

style_and_save(
    ax=ax,
    title='Matriz de Correlación',
    xlabel='Variables',
    ylabel='Variables',
    filename='matriz_de_correlacion.png',
    xtick_rotation=45,
    ytick_rotation=45
)

### 3.4.2. Eliminación de variables muy correlacionadas

In [ ]:
threshold = 0.7

abs_corr = correlation_matrix.abs()
high_corr_pairs = []

num_cols = abs_corr.shape[1]

for i in range(num_cols):
    for j in range(i + 1, num_cols):
        corr_value = abs_corr.iloc[i, j]
        if corr_value > threshold:
            high_corr_pairs.append((abs_corr.columns[i], abs_corr.columns[j], corr_value))

if high_corr_pairs:
    print(f"Parejas de variables con correlación mayor que {threshold}:")
    print(f"{'Variable 1':<20} {'Variable 2':<20} {'Correlación'}")
    print("-" * 55)
    for col1, col2, corr_value in high_corr_pairs:
        print(f"{col1:<20} {col2:<20} {corr_value:.2f}")
else:
    print(f"No hay parejas con correlación mayor que {threshold}")

In [ ]:
df = df.drop(columns=['Sha256', 'FileName', 'RegistryValueData', 'ApplicationName', 'OSVersion', 'City', 'State'])

In [ ]:
viewRemainingColumns()

## 3.5. Filtrado manual

In [ ]:
df = df.drop(columns=['OrgId', 'DetectorId', 'DeviceId', 'ApplicationId', 'OAuthApplicationId'])

In [ ]:
viewRemainingColumns()

## 3.6. Detección de anomalías
### 3.5.1. Eliminar valores extremos (outliers)
### 3.5.2. PCA

# 4. Entrenamiento

## 4.1. Preparación del entrenamiento

### 4.1.1. Importaciones

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

### 4.1.2. Codificación de la variable objetivo

In [ ]:
label_map = {
    'BenignPositive': 0,
    'FalsePositive': 1,
    'TruePositive': 2
}

### 4.1.3. División de datos

In [ ]:
# X e y
X = df.drop(columns=['IncidentGrade'])
y = df['IncidentGrade']

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(
#    X, y, test_size=0.2, random_state=42, stratify=y
#)

## 4.2. Modelos

In [ ]:
models = {
    'CatBoost': CatBoostClassifier(verbose=0),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'),
    'RandomForest': RandomForestClassifier()
}

## 4.3. Entrenamiento

In [ ]:
#for name, model in models.items():
#    print(f"\nEntrenando {name}...")
#    model.fit(X_train, y_train)
#    y_pred = model.predict(X_test)
#    print(f"\n{name} Classification Report:")
#    print(classification_report(y_test, y_pred))

## 4.4. Balanceo de clases

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import CondensedNearestNeighbour
from sklearn.utils import shuffle
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
from imblearn.under_sampling import ClusterCentroids
from sklearn.cluster import MiniBatchKMeans
from sklearn.utils import resample
import time

In [ ]:
target_size = y.value_counts()[1]

In [ ]:
# 1. Undersample clase 0 (BenignPositive)
rus_0 = RandomUnderSampler(sampling_strategy={0: target_size}, random_state=42)
X_rus_0, y_rus_0 = rus_0.fit_resample(X, y)

In [ ]:
# 2. Oversample clase 1 (FalsePositive) sobre el resultado del paso anterior
# smote = SMOTE(sampling_strategy={1: target_size}, random_state=42)
# X_balanced, y_balanced = smote.fit_resample(X_rus, y_rus)

In [ ]:
# 2. Undersample clase 2 (BenignPositive)
rus_2 = RandomUnderSampler(sampling_strategy={2: target_size}, random_state=42)
X_rus_final, y_rus_final = rus_2.fit_resample(X_rus_0, y_rus_0)

In [ ]:
# 3. Mezclar datos
X_balanced, y_balanced = shuffle(X_rus_final, y_rus_final, random_state=42)

In [ ]:
# 4. Verificar el resultado
print("Distribución tras undersampling:")
print(y_balanced.value_counts())

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_balanced, y_balanced, test_size=0.2, stratify=y_balanced, random_state=42
)

In [ ]:
#from imblearn.under_sampling import TomekLinks

#tl = TomekLinks(sampling_strategy='auto')  # auto aplica a todas las clases
#X_tomek, y_tomek = tl.fit_resample(X, y)

# Verificar tamaños
#from collections import Counter
#print("Distribución original:", Counter(y))
#print("Distribución tras Tomek:", Counter(y_tomek))

In [ ]:
# def extract_representatives_kmeans(X, y, n_samples_per_class=100):
#     X_rep = []
#     y_rep = []

#     for label in np.unique(y):
#         X_class = X[y == label]
#         y_class = y[y == label]

#         n_clusters = min(n_samples_per_class, len(X_class))

#         kmeans = KMeans(n_clusters=n_clusters, random_state=42)
#         kmeans.fit(X_class)
#         closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, X_class)

#         X_selected = X_class.iloc[closest]
#         y_selected = y_class.iloc[closest]

#         X_rep.append(X_selected)
#         y_rep.append(y_selected)

#     X_balanced_kmeans = pd.concat(X_rep)
#     y_balanced_kmeans = pd.concat(y_rep)

#     return X_balanced_kmeans, y_balanced_kmeans

In [ ]:
# Aplicar balanceo por representatividad
#X_kmeans, y_kmeans = extract_representatives_kmeans(X, y, n_samples_per_class=100)

# Dividir para entrenamiento
#X_train_km, X_test_km, y_train_km, y_test_km = train_test_split(
#    X_kmeans, y_kmeans, test_size=0.2, stratify=y_kmeans, random_state=42
#)

# Entrenar y evaluar modelos
#for name, model in models.items():
 #   print(f"\n{name} con KMeans representativo:")
 #  model.fit(X_train_km, y_train_km)
 # y_pred_km = model.predict(X_test_km)
 #print(classification_report(y_test_km, y_pred_km))

In [ ]:
# df_full = pd.concat([X, y], axis=1)

# df_sampled = df_full.groupby('IncidentGrade').apply(
#     lambda g: g.sample(n=min(len(g), 100000), random_state=42)
# ).reset_index(drop=True)

# X_sampled = df_sampled.drop(columns='IncidentGrade')
# y_sampled = df_sampled['IncidentGrade']

# mini_kmeans = MiniBatchKMeans(n_init=10, batch_size=1024, max_iter=100, random_state=42)

# cc = ClusterCentroids(estimator=mini_kmeans, random_state=42)

# X_cc, y_cc = cc.fit_resample(X_sampled, y_sampled)

# print("\nDistribución de clases tras ClusterCentroids:")
# print(pd.Series(y_cc).value_counts())

# # División en train y test
# X_train_cc, X_test_cc, y_train_cc, y_test_cc = train_test_split(
#     X_cc, y_cc, test_size=0.2, stratify=y_cc, random_state=42
# )

In [ ]:
# print("\nModelos tras balanceo:")

# for name, model in models.items():
#     print(f"\n✅ {name} con ClusterCentroids:")
#     model.fit(X_train_cc, y_train_cc)
#     y_pred_cnn = model.predict(X_test_cc)
#     print(f"\n{name} Classification Report:")
#     print(classification_report(y_test_cc, y_pred_cc))

In [ ]:
# for name, model in models.items():
#    print(f"\nEntrenando {name}...")
#    model.fit(X_train, y_train)
#    y_pred = model.predict(X_test)
#    print(f"\n{name} Classification Report:")
#    print(classification_report(y_test, y_pred))

## 4.5. Ensemble y stacking

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
ensemble = VotingClassifier(
    estimators=[
        ('catboost', models['CatBoost']),
        ('xgboost', models['XGBoost']),
        ('randomforest', models['RandomForest'])
    ],
    voting='soft'
)

ensemble.fit(X_train, y_train)

y_pred_ensemble = ensemble.predict(X_test)
print("\nEnsemble con Soft Voting sobre datos balanceados:")
print(classification_report(y_test, y_pred_ensemble))

In [ ]:
# stacking_clf = StackingClassifier(
#     estimators=[
#         ('catboost', models['CatBoost']),
#         ('xgboost', models['XGBoost']),
#         ('randomforest', models['RandomForest'])
#     ],
#     final_estimator=LogisticRegression(max_iter=1000),
#     cv=5,
#     n_jobs=-1,
#     passthrough=False
# )

# stacking_clf.fit(X_train, y_train)

# y_pred_stack = stacking_clf.predict(X_test)
# print("\nStacking con Logistic Regression como meta-modelo:")
# print(classification_report(y_test, y_pred_stack))

# 5. Evaluación

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import label_binarize
import numpy as np

In [ ]:
# Binarizar etiquetas para clasificación multiclase
classes = np.unique(y_test)
y_test_bin = label_binarize(y_test, classes=classes)
n_classes = y_test_bin.shape[1]

# Obtener probabilidades
y_proba_ensemble = ensemble.predict_proba(X_test)
# y_proba_stack = stacking_clf.predict_proba(X_test)

## 4.1. Curva ROC - AUC

In [ ]:
# Calcular curvas ROC y AUC por clase
fpr_ens, tpr_ens, fpr_stack, tpr_stack, auc_ens, auc_stack = {}, {}, {}, {}, {}, {}
for i in range(n_classes):
    fpr_ens[i], tpr_ens[i], _ = roc_curve(y_test_bin[:, i], y_proba_ensemble[:, i])
    # fpr_stack[i], tpr_stack[i], _ = roc_curve(y_test_bin[:, i], y_proba_stack[:, i])
    auc_ens[i] = auc(fpr_ens[i], tpr_ens[i])
    # auc_stack[i] = auc(fpr_stack[i], tpr_stack[i])

In [ ]:
# Plot ROC
plt.figure(figsize=(10, 6))
for i in range(n_classes):
    plt.plot(fpr_ens[i], tpr_ens[i], '--', label=f'Ensemble - Clase {classes[i]} (AUC = {auc_ens[i]:.2f})')
    # plt.plot(fpr_stack[i], tpr_stack[i], '-', label=f'Stacking - Clase {classes[i]} (AUC = {auc_stack[i]:.2f})')

plt.plot([0, 1], [0, 1], 'k--', label='Random')
plt.title('Curva ROC - Ensemble')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right')
plt.grid(True)
plt.tight_layout()
plt.show()

## 4.2. Matriz de confusión

In [ ]:
# Matriz de confusión - Ensemble
cm_ens = confusion_matrix(y_test, y_pred_ensemble)
disp_ens = ConfusionMatrixDisplay(confusion_matrix=cm_ens, display_labels=classes)
disp_ens.plot(cmap=plt.cm.Blues)
plt.title("Matriz de Confusión - Ensemble")
plt.show()

# Matriz de confusión - Stacking
# cm_stack = confusion_matrix(y_test, y_pred_stack)
# disp_stack = ConfusionMatrixDisplay(confusion_matrix=cm_stack, display_labels=classes)
# disp_stack.plot(cmap=plt.cm.Oranges)
# plt.title("Matriz de Confusión - Stacking")
# plt.show()

## 4.2. Precisión, Recall, F1-Score

## 4.4. Distribución de las probabilidades predichas por clase

In [ ]:
import seaborn as sns
import pandas as pd

In [ ]:
# Crear DataFrame con probabilidades predichas por el ensemble y stacking
ensemble_proba_df = pd.DataFrame(y_proba_ensemble, columns=[f'Clase {c}' for c in classes])
# stacking_proba_df = pd.DataFrame(y_proba_stack, columns=[f'Clase {c}' for c in classes])

# Añadir etiquetas verdaderas para distinguir las clases
ensemble_proba_df['Clase verdadera'] = y_test.values
# stacking_proba_df['Clase verdadera'] = y_test.values

In [ ]:
# Gráficas por clase
for c in classes:
    clase_str = f'Clase {c}'
    plt.figure(figsize=(10, 4))

    # Ensemble
    sns.kdeplot(
        data=ensemble_proba_df, x=clase_str, hue='Clase verdadera',
        common_norm=False, fill=True, alpha=0.4, linewidth=1.5,
        palette='tab10', label='Ensemble'
    )
    plt.title(f'Distribución de probabilidades - Ensemble ({clase_str})')
    plt.xlabel('Probabilidad estimada')
    plt.ylabel('Densidad')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # # Stacking
    # plt.figure(figsize=(10, 4))
    # sns.kdeplot(
    #     data=stacking_proba_df, x=clase_str, hue='Clase verdadera',
    #     common_norm=False, fill=True, alpha=0.4, linewidth=1.5,
    #     palette='tab10', label='Stacking'
    # )
    # plt.title(f'Distribución de probabilidades - Stacking ({clase_str})')
    # plt.xlabel('Probabilidad estimada')
    # plt.ylabel('Densidad')
    # plt.grid(True)
    # plt.tight_layout()
    # plt.show()